In [37]:
import pandas as pd
from langchain_community.llms import Ollama
from langchain.vectorstores import FAISS
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import faiss
import pprint
import warnings
warnings.filterwarnings("ignore")


In [7]:
df = pd.read_csv("customers-100.csv")
df

,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,DD37Cf93aecA6Dc,Sheryl,Baxter,Rasmussen Group,East Leonard,Chile,229.077.5154,397.884.0519x718,zunigavanessa@smith.info,2020-08-24,http://www.stephenson.com/
1,2,1Ef7b82A4CAAD10,Preston,Lozano,Vega-Gentry,East Jimmychester,Djibouti,5153435776,686-620-1820x944,vmata@colon.com,2021-04-23,http://www.hobbs.com/
2,3,6F94879bDAfE5a6,Roy,Berry,Murillo-Perry,Isabelborough,Antigua and Barbuda,+1-539-402-0259,(496)978-3969x58947,beckycarr@hogan.com,2020-03-25,http://www.lawrence.com/
3,4,5Cef8BFA16c5e3c,Linda,Olsen,"Dominguez, Mcmillan and Donovan",Bensonview,Dominican Republic,001-808-617-6467x12895,+1-813-324-8756,stanleyblackwell@benson.org,2020-06-02,http://www.good-lyons.com/
4,5,053d585Ab6b3159,Joanna,Bender,"Martin, Lang and Andrade",West Priscilla,Slovakia (Slovak Republic),001-234-203-0635x76146,001-199-446-3860x3486,colinalvarado@miles.net,2021-04-17,https://goodwin-ingram.com/
...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,cb8E23e48d22Eae,Karl,Greer,Carey LLC,East Richard,Guyana,(188)169-1674x58692,001-841-293-3519x614,hhart@jensen.com,2022-01-30,http://hayes-perez.com/
96,97,CeD220bdAaCfaDf,Lynn,Atkinson,"Ware, Burns and Oneal",New Bradview,Sri Lanka,+1-846-706-2218,605.413.3198,vkemp@ferrell.com,2021-07-10,https://novak-allison.com/
97,98,28CDbC0dFe4b1Db,Fred,Guerra,Schmitt-Jones,Ortegaland,Solomon Islands,+1-753-067-8419x7170,+1-632-666-7507x92121,swagner@kane.org,2021-09-18,https://www.ross.com/
98,99,c23d1D9EE8DEB0A,Yvonne,Farmer,Fitzgerald-Harrell,Lake Elijahview,Aruba,(530)311-9786,001-869-452-0943x12424,mccarthystephen@horn-green.biz,2021-08-11,http://watkins.info/


In [12]:
loader = CSVLoader("customers-100.csv")
docs = loader.load_and_split()

In [42]:
docs

[Document(metadata={'source': 'customers-100.csv', 'row': 0}, page_content='Index: 1\nCustomer Id: DD37Cf93aecA6Dc\nFirst Name: Sheryl\nLast Name: Baxter\nCompany: Rasmussen Group\nCity: East Leonard\nCountry: Chile\nPhone 1: 229.077.5154\nPhone 2: 397.884.0519x718\nEmail: zunigavanessa@smith.info\nSubscription Date: 2020-08-24\nWebsite: http://www.stephenson.com/'),
 Document(metadata={'source': 'customers-100.csv', 'row': 1}, page_content='Index: 2\nCustomer Id: 1Ef7b82A4CAAD10\nFirst Name: Preston\nLast Name: Lozano\nCompany: Vega-Gentry\nCity: East Jimmychester\nCountry: Djibouti\nPhone 1: 5153435776\nPhone 2: 686-620-1820x944\nEmail: vmata@colon.com\nSubscription Date: 2021-04-23\nWebsite: http://www.hobbs.com/'),
 Document(metadata={'source': 'customers-100.csv', 'row': 2}, page_content='Index: 3\nCustomer Id: 6F94879bDAfE5a6\nFirst Name: Roy\nLast Name: Berry\nCompany: Murillo-Perry\nCity: Isabelborough\nCountry: Antigua and Barbuda\nPhone 1: +1-539-402-0259\nPhone 2: (496)978-3

In [15]:
embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")

In [29]:
embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")
demo_embedding = embeddings.embed_query("")
index = faiss.IndexFlatL2(len(demo_embedding))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [30]:
vector_store.add_documents(documents=docs)

['525b5a49-f30b-4a54-b9fa-4f221b6cae7e',
 '59c469ea-a032-4b7a-8b52-15e180fe118c',
 '42593f56-e672-4291-8651-706fd3973c8e',
 '43bb7589-3209-43d4-b8fa-a2ce970ec4d6',
 '8c9e682c-61db-4e3b-9807-b6eb73c959a1',
 '3124e9d4-0d86-4178-b742-f795121028aa',
 '3f0f325f-0c68-4aa3-a2d5-d6b28ad94a02',
 '05101536-5d8c-4e9e-934c-8bce1714d805',
 '52a9378b-73ad-4a94-9dab-1945d4244589',
 '7d4c3e37-5cf1-4113-87cb-d69f7a594a1c',
 'a75e1753-db56-4f29-a092-dd299d804f77',
 '806e5954-8a66-4873-866d-0a7cae34c4b7',
 '669be35b-7a4c-4915-8671-310b93e5bd50',
 '141ecd98-c4ce-4879-a92a-9d5dd9ad3f7c',
 'c27d65a2-412b-4f93-8779-ac66ed8601f4',
 '990c448a-caa5-4619-9fcc-449e714d6b96',
 'f04050c4-6d7a-43e2-8285-37bcefcf6002',
 'ad262379-06b8-4306-a9fe-98b24c359c49',
 'd6418ae1-5e5b-4840-9502-592f828a9393',
 '5bcc8947-9823-47f9-b290-741905d3dc5e',
 '88680b25-1f86-42f9-bf16-f0c06dbf9b1a',
 'ed06f04e-114a-4d12-8f1c-32ce02e55f77',
 '70e916b5-bda5-4e7a-a04e-f763ad6ff0bd',
 '23baa42e-b662-495b-a3b4-422c4f09a90f',
 '44b81ed9-3a03-

In [32]:
prompt_template = """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer
    the question. If you don't know the answer, say that you
    don't know. Use three sentences maximum and keep the
    answer concise."
Context: {context}
Question: {question}
Answer: """

llm = Ollama(model="qwen2.5:3b")
retriver = vector_store.as_retriever()
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriver,
    chain_type_kwargs={"prompt": prompt}
)

In [38]:
pp = pprint.PrettyPrinter(width=80)
def chat(query):
    res= qa_chain.invoke(query)["result"]
    pp.pprint(res)

In [41]:
chat("what is Linda	Olsen's email?")

"Linda Olsen's email is stanleyblackwell@benson.org."
